In [97]:
import pandas as pd
import random

LEVEL_CONFIG = {1:2, 2:3, 3:4, 4:5, 5:5}

def decidir_accion(nivel, accuracy, tiempo_por_acierto, racha):
    # BAJAR si desempeño es claramente malo
    if accuracy < 0.6 and racha <= -2:
        return "BAJAR"

    # SUBIR si desempeño es claramente bueno
    if accuracy >= 0.95 and tiempo_por_acierto < (2.2 + nivel * 0.4) and racha >= 2:
        return "SUBIR"

    return "MANTENER"


data = []
racha = 0

for _ in range(6000):

    nivel = random.randint(1, 5)
    largo = LEVEL_CONFIG[nivel]

    # Errores más realistas
    max_errores = min(largo, random.choice([1, 2]) if nivel >= 3 else 1)
    errores = random.randint(0, max_errores)

    aciertos = max(0, largo - errores)
    accuracy = aciertos / largo if largo > 0 else 0

    # Tiempo realista
    if errores == 0:
        tiempo_total = random.uniform(largo * 1.1, largo * 2.0)
    else:
        tiempo_total = random.uniform(largo * 1.8, largo * 3.5)

    tiempo_por_acierto = tiempo_total / max(1, aciertos)

    # Racha
    if errores == 0:
        racha += 1
    else:
        racha -= 1
    racha = max(-5, min(5, racha))

    accion = decidir_accion(nivel, accuracy, tiempo_por_acierto, racha)

    data.append([
        nivel, largo, aciertos, errores,
        round(accuracy, 2),
        round(tiempo_por_acierto, 2),
        racha,
        accion
    ])

df = pd.DataFrame(data, columns=[
    "Nivel", "Largo_Patron", "Aciertos", "Errores",
    "Accuracy", "Tiempo_por_Acierto", "Racha", "Accion"
])

df.to_csv("dataset_simon.csv", index=False)

df


,Nivel,Largo_Patron,Aciertos,Errores,Accuracy,Tiempo_por_Acierto,Racha,Accion
0,5,5,4,1,0.8,3.32,-1,MANTENER
1,4,5,4,1,0.8,3.92,-2,MANTENER
2,1,2,2,0,1.0,1.28,-1,MANTENER
3,3,4,2,2,0.5,6.43,-2,BAJAR
4,1,2,2,0,1.0,1.72,-1,MANTENER
...,...,...,...,...,...,...,...,...
5995,5,5,5,0,1.0,1.48,-4,MANTENER
5996,2,3,3,0,1.0,1.31,-3,MANTENER
5997,1,2,1,1,0.5,5.35,-4,BAJAR
5998,3,4,4,0,1.0,1.41,-3,MANTENER


In [98]:
import pandas as pd
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import train_test_split
import joblib

df = pd.read_csv("dataset_simon.csv")

X = df.drop(columns=["Accion"])
y = df["Accion"]

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2)

modelo = RandomForestClassifier(n_estimators=200, max_depth=10)
modelo.fit(X_train, y_train)

print("Accuracy:", modelo.score(X_test, y_test))

joblib.dump(modelo, "modelo_rf_new.pkl")
print("Modelo guardado.")


Accuracy: 1.0
Modelo guardado.


In [110]:
#['Nivel', 'Largo_Patron', 'Aciertos', 'Errores', 'Tiempo_Reaccion', 'Racha']
import numpy as np

test = pd.DataFrame([{
    'Nivel': 3,
    'Largo_Patron' : 4,
    'Aciertos': 3,
    'Errores': 1,
    'Accuracy' : 1,
    'Tiempo_por_Acierto': 10,
    'Racha': 4
}])
res = modelo.predict(test)
print(test,res)

   Nivel  Largo_Patron  Aciertos  Errores  Accuracy  Tiempo_por_Acierto  Racha
0      3             4         3        1         1                  10      4 ['MANTENER']


In [34]:
import pandas as pd
import random
import numpy as np

# Configuración del juego
LEVEL_CONFIG = {1: 2, 2: 3, 3: 4, 4: 5, 5: 6}

def decidir_accion_flexible(nivel, accuracy, tiempo_promedio, racha):
    """
    Lógica ajustada:
    - Muy difícil BAJAR (solo desastres o rachas terribles).
    - Difícil SUBIR (requiere racha sólida).
    - Gran zona de MANTENER para estabilidad.
    """
    
    # Umbral de tiempo para considerar subir (se vuelve más estricto en niveles altos)
    umbral_tiempo_subir = 2.0 - (nivel * 0.12) # Suavizado un poco (0.15 -> 0.12)
    
    # --- CRITERIOS PARA BAJAR (Más flexibles) ---
    
    # 1. Desastre Total: Menos del 40% de aciertos.
    # Aquí no importa la racha, si fallas tanto, el nivel es muy alto para ti.
    if accuracy <= 0.4:
        return "BAJAR"
    
    # 2. Desempeño Pobre (40% - 60%) + Mala Racha
    # Si aciertas la mitad Y vienes perdiendo (racha negativa), bajas.
    # PERO, si tienes racha positiva (tuviste un mal round), te MANTENER.
    if accuracy <= 0.6 and racha < 0:
        return "BAJAR"
        
    # 3. Errores Leves (60% - 85%) + Racha Pésima
    # Solo bajas si fallas un poco Y tu racha es terrible (-3 o peor).
    # Esto evita que bajes por un simple error de dedo si tu historial es decente.
    if accuracy < 0.85 and racha <= -3:
        return "BAJAR"

    # --- CRITERIOS PARA SUBIR (Con buena racha) ---
    
    # Solo consideramos subir si es PERFECTO
    if accuracy == 1.0:
        es_rapido = tiempo_promedio <= umbral_tiempo_subir
        
        # Racha Progresiva: Exigimos más "pruebas" en niveles altos
        if nivel == 1:
            racha_necesaria = 2
        elif nivel == 2:
            racha_necesaria = 3
        else:
            racha_necesaria = 4 # Niveles 3, 4, 5 requieren solidez
            
        if racha >= racha_necesaria and es_rapido:
            return "SUBIR"

    # --- POR DEFECTO: MANTENER ---
    # Esto cubre:
    # - Aciertos perfectos pero lentos.
    # - Aciertos perfectos pero sin suficiente racha.
    # - Errores leves con racha normal (-1, -2).
    return "MANTENER"

data = []

# Generamos 20,000 muestras para asegurar cobertura de casos borde
for _ in range(20000):
    
    nivel = random.randint(1, 5)
    largo_patron = LEVEL_CONFIG[nivel]

    # --- Generación de Errores (Ponderada) ---
    posibles_errores = list(range(largo_patron + 1))
    
    # Simulamos un jugador decente:
    # 65% Perfecto, 20% 1 Error, 10% 2 Errores, 5% Desastre
    pesos_base = [0.65, 0.20, 0.10]
    
    cantidad_extra = len(posibles_errores) - 3
    if cantidad_extra > 0:
        peso_extra = 0.05 / cantidad_extra
        pesos = pesos_base + [peso_extra] * cantidad_extra
    else:
        pesos = pesos_base[:len(posibles_errores)]
        
    errores = random.choices(posibles_errores, weights=pesos, k=1)[0]
    aciertos = largo_patron - errores
    accuracy = round(aciertos / largo_patron, 2)
    
    # --- Simulación de Tiempo ---
    base_time = random.uniform(0.9, 1.7)
    if errores > 0: base_time += random.uniform(0.5, 1.5)
    tiempo_promedio = round(base_time, 2)
    
    # --- Simulación de Racha (-4 a +5) ---
    racha_actual = random.randint(-4, 5)
    
    # Etiquetar con la nueva lógica flexible
    accion = decidir_accion_flexible(nivel, accuracy, tiempo_promedio, racha_actual)
    
    data.append([nivel, accuracy, tiempo_promedio, racha_actual, accion])

# Crear DataFrame y Guardar
df = pd.DataFrame(data, columns=["Nivel", "Accuracy", "Tiempo_Promedio", "Racha", "Accion"])

print("Distribución de acciones (MANTENER debería dominar):")
print(df['Accion'].value_counts(normalize=True))

df.to_csv("dataset_simon_flexible.csv", index=False)
print("\nDataset generado: 'dataset_simon_flexible.csv'")

Distribución de acciones (MANTENER debería dominar):
Accion
MANTENER    0.70220
SUBIR       0.15435
BAJAR       0.14345
Name: proportion, dtype: float64

Dataset generado: 'dataset_simon_flexible.csv'


In [35]:
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import classification_report
import joblib

# 1. Cargar el dataset mejorado
df = pd.read_csv("dataset_simon_flexible.csv")

# 2. Separar Features (X) y Target (y)
# OJO: No incluyas 'Aciertos' o 'Errores' brutos si ya tienes 'Accuracy', 
# ya que están muy correlacionados. Usa las métricas relativas.
X = df[["Nivel", "Accuracy", "Tiempo_Promedio", "Racha"]]
y = df["Accion"]

# 3. Split
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# 4. Crear el modelo
# n_estimators=100: crea 100 árboles y votan entre ellos (da estabilidad)
# max_depth=10: evita que el árbol se aprenda de memoria los datos raros (overfitting)
clf = RandomForestClassifier(n_estimators=100, max_depth=10, random_state=42)

# 5. Entrenar
clf.fit(X_train, y_train)

# 6. Evaluar
print("Reporte de Calidad del Modelo:")
y_pred = clf.predict(X_test)
print(classification_report(y_test, y_pred))

# 7. GUARDAR EL MODELO PARA EL BACKEND
# Esto genera un archivo binario que contiene el cerebro entrenado
joblib.dump(clf, 'modelo_simon_v1.pkl')
print("Modelo guardado como 'modelo_simon_v2.pkl'")

Reporte de Calidad del Modelo:
              precision    recall  f1-score   support

       BAJAR       1.00      1.00      1.00       578
    MANTENER       1.00      1.00      1.00      2814
       SUBIR       1.00      1.00      1.00       608

    accuracy                           1.00      4000
   macro avg       1.00      1.00      1.00      4000
weighted avg       1.00      1.00      1.00      4000

Modelo guardado como 'modelo_simon_v2.pkl'


In [50]:
import pandas as pd
import numpy as np
from sklearn.model_selection import StratifiedKFold, cross_val_score
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import classification_report, make_scorer, f1_score
import joblib

# 1. Cargar el dataset flexible
df = pd.read_csv("dataset_simon_flexible.csv")

# 2. Separar Features (X) y Target (y)
X = df[["Nivel", "Accuracy", "Tiempo_Promedio", "Racha"]]
y = df["Accion"]

# 3. Configurar K-Fold (Validación Cruzada)
# n_splits=5: Dividirá los datos en 5 partes. Entrena con 4 y prueba con 1, rotando 5 veces.
# shuffle=True: Mezcla los datos antes de dividir (fundamental).
skf = StratifiedKFold(n_splits=5, shuffle=True, random_state=42)

# 4. Crear la configuración del modelo
clf_k = RandomForestClassifier(n_estimators=100, max_depth=10, random_state=42)

print(f"--- Iniciando Validación Cruzada (K-Fold = 5) ---")

# 5. Evaluación Robusta
# Calculamos la precisión (accuracy) y el F1-Score (mejor para clases desbalanceadas)
scores_acc = cross_val_score(clf_k, X, y, cv=skf, scoring='accuracy')
# Usamos 'weighted' porque tienes 3 clases (SUBIR, BAJAR, MANTENER) que pueden no estar balanceadas
scores_f1 = cross_val_score(clf_k, X, y, cv=skf, scoring='f1_weighted') 

print(f"Resultados por Fold (Accuracy): {scores_acc}")
print(f"Accuracy Promedio: {np.mean(scores_acc):.4f} (+/- {np.std(scores_acc):.4f})")
print(f"F1-Score Promedio: {np.mean(scores_f1):.4f}")

# 6. Entrenamiento FINAL para Producción
# Una vez validado que el modelo es estable (poca desviación estándar),
# lo entrenamos con EL 100% DE LOS DATOS para que sea lo más inteligente posible.
print("\n--- Entrenando Modelo Final para Producción ---")
clf_k.fit(X, y)

# Opcional: Verificar con un reporte sobre todo el set (solo informativo)
# Nota: Esto será optimista porque ya vio los datos, pero sirve para confirmar que aprendió.
print(classification_report(y, clf_k.predict(X)))

# 7. GUARDAR EL MODELO
nombre_archivo = 'modelo_simon_v3.pkl'
joblib.dump(clf, nombre_archivo)
print(f"Modelo final guardado exitosamente como '{nombre_archivo}'")

--- Iniciando Validación Cruzada (K-Fold = 5) ---
Resultados por Fold (Accuracy): [1. 1. 1. 1. 1.]
Accuracy Promedio: 1.0000 (+/- 0.0000)
F1-Score Promedio: 1.0000

--- Entrenando Modelo Final para Producción ---
              precision    recall  f1-score   support

       BAJAR       1.00      1.00      1.00      2869
    MANTENER       1.00      1.00      1.00     14044
       SUBIR       1.00      1.00      1.00      3087

    accuracy                           1.00     20000
   macro avg       1.00      1.00      1.00     20000
weighted avg       1.00      1.00      1.00     20000

Modelo final guardado exitosamente como 'modelo_simon_v3.pkl'


In [36]:
#Nivel,Accuracy,Tiempo_Promedio,Racha,Accion_Final,Errores_Reales
#3,1.0,3.77,5,BAJAR,0
test = pd.DataFrame([{
    'Nivel': 4,
    'Accuracy' : .8,
    'Tiempo_Promedio': .95,
    'Racha': -2
}])
res = clf.predict(test)
print(test,res)

   Nivel  Accuracy  Tiempo_Promedio  Racha
0      4       0.8             0.95     -2 ['MANTENER']


In [51]:
df_test_pasado = pd.read_csv('ml_input_log_new.csv')
df_test_new = df_test_pasado.drop(columns={'Accion_Final','Errores_Reales'})
res_new = clf_k.predict(df_test_new)
df_test_pasado['new_predict'] = res_new

In [56]:
df_test_pasado[df_test_pasado['Nivel'] == 3].head(15)

,Nivel,Accuracy,Tiempo_Promedio,Racha,Accion_Final,Errores_Reales,new_predict
8,3,1.00,1.81,4,MANTENER,0,MANTENER
9,3,1.00,1.63,5,MANTENER,0,SUBIR
10,3,0.00,4.36,-1,BAJAR,1,BAJAR
22,3,1.00,1.46,5,SUBIR,0,SUBIR
24,3,1.00,1.61,1,MANTENER,0,MANTENER
25,3,0.00,3.73,-1,BAJAR,1,BAJAR
45,3,0.00,4.70,-1,BAJAR,1,BAJAR
47,3,0.75,2.46,-1,BAJAR,1,MANTENER
60,3,1.00,1.69,1,MANTENER,0,MANTENER
61,3,1.00,1.66,2,MANTENER,0,MANTENER
